Считывание и инициализация.

In [6]:
import warnings  
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier



X_train = pd.read_csv('Train.csv')
X_test = pd.read_csv('Test.csv')


print(X_train['street_id'].nunique())
print(X_train['area'].max())


672
217


Работа с фичами.

In [7]:
test_index=X_test['id'].values
print(test_index)
y=X_train.price
X_train.drop(['price'], axis=1, inplace=True)
def onehot(df,n,name):
    for i in range(n):
        df[name+str(i)]=df[name].apply(lambda x: 1 if x==i else 0)
    return df
def featureProcessing(df):
    df.drop(['id'], axis=1, inplace=True)
    df['Y']=df['date'].apply(lambda x: int(x.split('-')[0]))
    df['M']=df['date'].apply(lambda x: int(x.split('-')[1]))
    df.drop(['date'], axis=1, inplace=True)
    df=onehot(df,250,'area')
    df.drop(['area'], axis=1, inplace=True)
    return df
X_train=featureProcessing(X_train)
X_test=featureProcessing(X_test)

[100000 100001 100002 ... 199997 199998 199999]


Обучение модели

In [10]:
model_XGB = XGBRegressor(n_estimators=5000, learning_rate=0.055, objective="reg:squarederror", n_jobs=-1, random_state=0)

model_XGB.fit(X_train, np.log1p(y))

ensemble_preds = model_XGB.predict(X_test)

In [11]:
preds = np.expm1(ensemble_preds)

output = pd.DataFrame({'id': test_index,
                      'price': preds})

output.to_csv('submission.csv', index=False)